<a href="https://colab.research.google.com/github/jessica-aaao/ChordsExtractor/blob/main/ChordExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install -q -U "yt-dlp[default]" --upgrade
!pip install -q -U openai-whisper
!pip install -q -U demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 42.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.6 MB/s eta 

In [2]:
import requests
import whisper
import json
import pandas as pd

In [3]:
from google.colab import drive
from IPython.display import display
from bs4 import BeautifulSoup

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


# Common

In [5]:
class SongUrls:
    def __init__(self, name, audio, lyrics, chords):
        self.name = name
        self.audio = audio
        self.lyrics = lyrics
        self.chords = chords

    def get_name(self):
        return self.name

    def get_audio_url(self):
        return self.audio

    def get_lyrics_url(self):
        return self.lyrics

    def get_chords_url(self):
        return self.chords

In [37]:
def get_urls():
    print(f'Fetching urls...\n\n')

    file_path = '/content/drive/My Drive/TCC/CodeData/songs.csv'
    songs = pd.read_csv(file_path)

    print(f'Urls fetched!\n\n')

    return songs

In [38]:
def get_songs_from_csv():
    songs = get_urls()

    song_urls = []

    print(f'Creating SongUrls...\n\n')

    for index, row in songs.iterrows():
        song_name = row['Song Name']
        audio_url = row['Audio URL']
        lyrics_url = row['Lyrics URL']
        chords_url = row['Chords URL']

        song_urls.append(SongUrls(song_name, audio_url, lyrics_url, chords_url))

    print(f'SongUrls created!\n\n')

    return song_urls


In [8]:
def save_to_file(data, path):
    with open(path, 'w') as file:
        file.write(data)

    print(f'Saved as {path}')

# Sound

In [39]:
def extract_voice_from_audio(audio_path, song_name):
    print(f'Extracting vocals from {audio_path}...\n\n')

    !demucs --mp3 --two-stems=vocals {audio_path} -o vocals/

    return f"vocals/htdemucs/{song_name}/vocals.mp3"


In [40]:
def extract_sound_recording(youtube_url, song_name):
    output_path = f"audios/{song_name}"

    !yt-dlp -q -x --audio-format 'mp3' -o {output_path} {youtube_url}

    print(f'Audio saved as {output_path}.mp3!\n\n')

    vocals_path = extract_voice_from_audio(f"{output_path}.mp3", song_name)

    print(f'Vocals saved as {vocals_path}!\n\n')

    return vocals_path

# Lyrics

## Lyrics Extraction from Webpage

In [41]:
def extract_lyrics_from_html(html):
    print(f'Fetching lyrics...!\n\n')

    lyricsTag = html.find('div', class_='lyric-original')
    lyrics = ""

    for p in lyricsTag.find_all('p'):
        for br in p.find_all('br'):
            br.replace_with('\n')
        lyrics += p.get_text() + "\n"

    print(f'Lyrics fetched!\n\n')

    return lyrics


In [43]:
def get_lyrics_from_webpage(lyric_url):
    print(f'Fetching webpage {lyric_url}...\n\n')
    response = requests.get(lyric_url)

    if response.status_code == 200:
        print(f'Webpage fetched!\n\n')
        htmlContent = BeautifulSoup(response.content, 'html.parser')

        lyrics = extract_lyrics_from_html(htmlContent)
        # save_lyrics_to_file(lyrics)

        return lyrics.splitlines()
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}\n\n")

## Lyrics Sync to Audio

In [50]:
def adjust_lyrics(transcribed_segments, comparison_lyrics):
    print(f"Adjusting lyrics...\n\n")

    transcribed_lyrics = transcribed_segments

    for index, segment in enumerate(transcribed_lyrics):
        try:
            line = comparison_lyrics[index]

            if segment['text'].lower() != line.lower():
                segment['text'] = line

            words = line.split()

            for i, word in enumerate(words):
                try:
                    transcribed_word = segment['words'][i]['word']

                    if transcribed_word.lower().strip() != word.lower():
                        print(f"Word mismatch: {transcribed_word} != {word}")

                        segment['words'][i]['word'] = word
                except:
                    print(f"Word not found: {word}")
                    continue
        except:
            print(f"Line not found: {segment['text']}")
            continue

    print(f"Lyrics adjusted:\n{json.dumps(transcribed_lyrics, indent=2)}\n\n")

    return transcribed_lyrics

In [45]:
def transcribe_audio(audio_path):
    print(f"Transcribing audio...\n\n")

    model = whisper.load_model("turbo", device='cuda')
    result = model.transcribe(audio_path, word_timestamps=True)

    print(f"Audio Transcribed!\n\n")

    return result['segments']


In [44]:
def get_synced_lyrics(lyric_url, audio_path):
    print(f"Getting synced lyrics...\n\n")

    transcribed_lyrics = transcribe_audio(audio_path)
    comparison_lyrics = get_lyrics_from_webpage(lyric_url)
    adjusted_lyrics = adjust_lyrics(transcribed_lyrics, comparison_lyrics)

    print(f"Synced lyrics ready!\n\n")

    return adjusted_lyrics

# Chords

## Chords Extraction from Audio

In [ ]:
def get_synced_chords:
    pass

## Chords Sync to Lyrics



---



# MAIN

## Inputs

In [ ]:
song_url = input("Enter youtube url: ")
# lyric_url= input("Enter lyric url: ")



In [ ]:
extract_sound_recording(song_url, "nome")

## From CSV

In [51]:
songs = get_songs_from_csv()

for song in songs:
    song_name = song.get_name()
    print(f"Generating {song_name} chord sheet...\n\n")
    vocals_path = extract_sound_recording(song.get_audio_url(), song_name)
    synced_lyrics = get_synced_lyrics(song.get_lyrics_url(), vocals_path)
    # get_synced_chords(song.get_chords_url())

Fetching urls...


Urls fetched!


Creating SongUrls...


SongUrls created!


Generating Lisboa chord sheet...


Audio saved as audios/Lisboa.mp3!


Extracting vocals from audios/Lisboa.mp3...


Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/vocals/htdemucs
Separating track audios/Lisboa.mp3
100%|██████████████████████████████████████████████| 228.14999999999998/228.14999999999998 [00:12<00:00, 17.86seconds/s]
Vocals saved as vocals/htdemucs/Lisboa/vocals.mp3!


Getting synced lyrics...


Transcribing audio...




/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Audio Transcribed!


Fetching webpage https://www.letras.mus.br/anavitoria/lisboa-part-lenine/...


Webpage fetched!


Fetching lyrics...!


Lyrics fetched!


Adjusting lyrics...


Word mismatch:  mim != mim,
Word mismatch:  morto, != morto
Word mismatch:  claro != ou
Word not found: Claro
Word mismatch:  o != meu
Word mismatch:  meu != melhor
Word mismatch:  que != o
Word mismatch:  eu != que
Word mismatch:  quero != eu
Word mismatch:  ouvir != quero
Word not found: ouvir
Word mismatch:  cara != cara,
Word mismatch:  e != o
Word mismatch:  mim != mim,
Word mismatch:  morto, != morto
Word mismatch:  claro != ou
Word not found: Claro
Word mismatch:  o != meu
Word mismatch:  meu != melhor
Word mismatch:  que != o
Word mismatch:  eu != que
Word mismatch:  quero != eu
Word mismatch:  ouvir != quero
Word not found: ouvir
Word not found: (me
Word not found: diga
Word not found: agora,
Word not found: por
Word not found: favor)
Word mismatch:  Me != Que
Word mismatch:  diga != eu
Word mismatc

In [53]:
text1 = "Hello world! Hello Python!"
text2 = "Hello, world! Hello, Java!"

# Split the strings into words
words1 = text1.split()
words2 = text2.split()

# Compare word by word
for word1, word2 in zip(words1, words2):
    if word1 != word2:
        print(f"Difference: {word1} != {word2}")

Difference: Hello != Hello,
Difference: Hello != Hello,
Difference: Python! != Java!
